In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/tweet-sentiment-extraction/train.csv
/kaggle/input/tweet-sentiment-extraction/test.csv
/kaggle/input/tweet-sentiment-extraction/sample_submission.csv


In [2]:
#importing required packages
import re,nltk
from nltk.corpus import stopwords
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.tokenize import word_tokenize
import spacy
from nltk.sentiment.vader import SentimentIntensityAnalyzer

/opt/conda/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [3]:
#downloading the required packages from nltk downloader
nltk.download(['punkt','stopwords','vader_lexicon'])

[nltk_data] <urlopen error [Errno -3] Temporary failure in name
[nltk_data]     resolution>


False

In [4]:
#defining a function to read and preprocess data
def preprocessor(text):
    #using regex to remove http from the dataframe
    text=str(text).lower()
    text=re.sub('http\S+','',text)
    return text

In [5]:
#reading the trainset
trainset=pd.read_csv('/kaggle/input/tweet-sentiment-extraction/train.csv')

In [6]:
#obtaining cleaned text
trainset['cleaned_data']=trainset['text'].apply(preprocessor)

In [7]:
#viewing trainset's first 5 rows
trainset.head()

,textID,text,selected_text,sentiment,cleaned_data
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"i`d have responded, if i were going"
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,sooo sad i will miss you here in san diego!!!
2,088c60f138,my boss is bullying me...,bullying me,negative,my boss is bullying me...
3,9642c003ef,what interview! leave me alone,leave me alone,negative,what interview! leave me alone
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"sons of ****, why couldn`t they put them on t..."


In [8]:
#dropping the original text
trainset.drop(['text'],1,inplace=True)

In [9]:
trainset.head()

,textID,selected_text,sentiment,cleaned_data
0,cb774db0d1,"I`d have responded, if I were going",neutral,"i`d have responded, if i were going"
1,549e992a42,Sooo SAD,negative,sooo sad i will miss you here in san diego!!!
2,088c60f138,bullying me,negative,my boss is bullying me...
3,9642c003ef,leave me alone,negative,what interview! leave me alone
4,358bd9e861,"Sons of ****,",negative,"sons of ****, why couldn`t they put them on t..."


In [10]:
#function for Sentiment Intensity Analyser and returning the processed_text
def polarity_determiner(df_process):
    train_data = df_process['cleaned_data']
    train_data_sentiment = df_process['sentiment']
    #initialising a list that contains all the processed text
    selected_text_processed = []
    #initialising the Sentiment Intensity Analyser
    #this will determine the polarity of each word
    analyser = SentimentIntensityAnalyzer()
    for j in range(0 , len(train_data)):
        #using regex to remove http from the train_data
        text = re.sub(r'http\S+', '', str(train_data.iloc[j]))
        #for neutral similarity, all text is appended 
        if(train_data_sentiment.iloc[j] == "neutral" or len(text.split()) < 2):
            selected_text_processed.append(str(text))
        #for sentiments like positive and negative, only words that have the highest polarity are determined as these words strongly determine the sentiment
        if(train_data_sentiment.iloc[j] == "positive" and len(text.split()) >= 2):
            aa = re.split(' ', text)
        
            ss_arr = ""
            #assigning an initial polarity of 0
            polar = 0
            for qa in range(0,len(aa)):
                score = analyser.polarity_scores(aa[qa])
                if score['compound'] >polar:
                    polar = score['compound']
                    ss_arr = aa[qa]
            if len(ss_arr) != 0:
                selected_text_processed.append(ss_arr)   
            if len(ss_arr) == 0:
                selected_text_processed.append(text)
        #repeating the same case for negative sentiment
        if(train_data_sentiment.iloc[j] == "negative"and len(text.split()) >= 2):
            aa = re.split(' ', text)
        
            ss_arr = ""
            polar = 0
            for qa in range(0,len(aa)):
                score = analyser.polarity_scores(aa[qa])
                if score['compound'] <polar:
                    polar = score['compound']
                    ss_arr = aa[qa]
            if len(ss_arr) != 0:
                selected_text_processed.append(ss_arr)   
            if len(ss_arr) == 0:
                selected_text_processed.append(text)  
    return selected_text_processed

In [11]:
selected_train=polarity_determiner(trainset)

In [12]:
len(selected_train)

27481

In [13]:
#reading the testset into a dataframe
testset=pd.read_csv('/kaggle/input/tweet-sentiment-extraction/test.csv')

In [14]:
#same for test dataset
testset['cleaned_data']=testset['text'].apply(preprocessor)

In [15]:
selected_test=polarity_determiner(testset)

In [16]:
text_id=testset['textID']

In [17]:
idlist=[text_id.iloc[i] for i in range(len(text_id))]

In [18]:
df_sub=pd.DataFrame({'textID':idlist,'selected_text':selected_test})

In [19]:
df_sub.head()

,textID,selected_text
0,f87dea47db,last session of the day
1,96d74cb729,exciting
2,eee518ae67,shame!
3,01082688c6,happy
4,33987a8ee5,like


In [20]:
df_sub.to_csv('/kaggle/working/submission.csv',index=False)